In [9]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!pip install mediapipe opencv-python

In [4]:
#2

import cv2
import mediapipe as mp
import os
import random
from shutil import copyfile

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)  # Use static image mode for processing single images

# Define valid image file extensions
valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp')

# Define a mapping for yoga poses to Class IDs
pose_classes = {
    "aNoPose": 4,

}

# Define the directory paths for images
dataset_path = '/content/drive/MyDrive/KMUTT/AI/onlyNoPose-Dataset'  # Folder containing all pose images
save_dir = '/content/drive/MyDrive/KMUTT/AI/NoPoseAnnotated'  # Base folder to save training, validation, and test datasets

# YOLOv8 keypoint indices based on the mapping from MediaPipe to COCO's 17 keypoints
keypoint_mapping = [
    0,  # Nose
    2,  # Left Eye Inner / Left Eye
    5,  # Right Eye Inner / Right Eye
    7,  # Left Ear
    8,  # Right Ear
    11, # Left Shoulder
    12, # Right Shoulder
    13, # Left Elbow
    14, # Right Elbow
    15, # Left Wrist
    16, # Right Wrist
    23, # Left Hip
    24, # Right Hip
    25, # Left Knee
    26, # Right Knee
    27, # Left Ankle
    28  # Right Ankle
]

# Function to process and save annotations for a given yoga pose
def annotate_pose(image_path, pose_type, save_img_path, save_label_path):
    # Load the image
    image = cv2.imread(image_path)

    if image is None:
        print(f"Failed to load image: {image_path}")
        return

    # Get image dimensions
    img_height, img_width = image.shape[:2]

    # Convert the image to RGB (MediaPipe expects RGB input)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detect human body pose using MediaPipe
    results = pose.process(image_rgb)

    if not results.pose_landmarks:
        print(f"No pose detected in {image_path}")
        return

    # Retrieve keypoints from the detected pose landmarks (33 keypoints from MediaPipe)
    keypoints = results.pose_landmarks.landmark

    # Map the 33 keypoints to the 17 keypoints for YOLOv8 using the keypoint_mapping
    yolo_keypoints = [keypoints[i] for i in keypoint_mapping]

    # Get x and y coordinates of the selected 17 keypoints
    x_coords = [kp.x for kp in yolo_keypoints]
    y_coords = [kp.y for kp in yolo_keypoints]
    x_min, x_max = min(x_coords), max(x_coords)
    y_min, y_max = min(y_coords), max(y_coords)

    # Calculate the bounding box (center_x, center_y, width, height) and normalize it
    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2
    bbox_width = (x_max - x_min)
    bbox_height = (y_max - y_min)

    # Normalize keypoints and add visibility flag using MediaPipe's visibility flag
    normalized_keypoints = []
    for kp in yolo_keypoints:
        kp_x_normalized = kp.x  # Already normalized (relative to image width)
        kp_y_normalized = kp.y  # Already normalized (relative to image height)
        visibility_flag = kp.visibility  # Use MediaPipe's visibility value
        normalized_keypoints.extend([kp_x_normalized, kp_y_normalized, visibility_flag])

    # Get the class ID for the given pose type
    class_id = pose_classes.get(pose_type, -1)
    if class_id == -1:
        print(f"Unknown pose type: {pose_type}")
        return

    # Construct YOLOv8 annotation line (class ID, bbox, and keypoints)
    annotation_line = f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height} " + " ".join(map(str, normalized_keypoints))

    # Save the annotation in YOLOv8 format to a .txt file
    with open(save_label_path, 'w') as f:
        f.write(annotation_line)

    # Save the original image (without drawing pose landmarks)
    copyfile(image_path, save_img_path)
    print(f"Original image saved at {save_img_path} and annotation saved at {save_label_path}")

# Function to split dataset into train, validation, and test sets
def split_dataset(image_paths):
    random.shuffle(image_paths)
    total_images = len(image_paths)

    # Calculate the split sizes
    train_size = int(0.6 * total_images)
    val_size = int(0.2 * total_images)
    test_size = total_images - train_size - val_size

    # Split the dataset
    train_images = image_paths[:train_size]
    val_images = image_paths[train_size:train_size + val_size]
    test_images = image_paths[train_size + val_size:]

    return train_images, val_images, test_images

# Get all image files for each yoga pose
all_images = {
    'aNoPose': [f"{dataset_path}/aNoPose/{img}" for img in os.listdir(f"{dataset_path}/aNoPose") if img.lower().endswith(valid_extensions)],
    # 'easyseat': [f"{dataset_path}/moreeasyseat/{img}" for img in os.listdir(f"{dataset_path}/moreeasyseat") if img.lower().endswith(valid_extensions)],
    # 'tree': [f"{dataset_path}/moretree/{img}" for img in os.listdir(f"{dataset_path}/moretree") if img.lower().endswith(valid_extensions)],
    # 'triangle': [f"{dataset_path}/moretriangle/{img}" for img in os.listdir(f"{dataset_path}/moretriangle") if img.lower().endswith(valid_extensions)],
}

# Define the base directories for images and labels
base_image_dir = os.path.join(save_dir, 'images')
base_label_dir = os.path.join(save_dir, 'labels')

# Create subdirectories for training and validation within the images and labels directories
os.makedirs(os.path.join(base_image_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(base_image_dir, 'val'), exist_ok=True)
os.makedirs(os.path.join(base_label_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(base_label_dir, 'val'), exist_ok=True)

# Process each pose type and split the dataset
for pose_type, images in all_images.items():
    # Split dataset
    train_images, val_images, test_images = split_dataset(images)

    # Annotate and save images and labels in train and val sets
    for img_path in train_images:
        img_basename = os.path.basename(img_path)
        save_img_path = os.path.join(base_image_dir, 'train', img_basename)
        save_label_path = os.path.join(base_label_dir, 'train', os.path.splitext(img_basename)[0] + '.txt')
        annotate_pose(img_path, pose_type, save_img_path, save_label_path)

    for img_path in val_images:
        img_basename = os.path.basename(img_path)
        save_img_path = os.path.join(base_image_dir, 'val', img_basename)
        save_label_path = os.path.join(base_label_dir, 'val', os.path.splitext(img_basename)[0] + '.txt')
        annotate_pose(img_path, pose_type, save_img_path, save_label_path)

    # Save only images for the test set, without annotations
    test_image_dir = os.path.join(base_image_dir, 'test')
    os.makedirs(test_image_dir, exist_ok=True)
    for img_path in test_images:
        save_img_path = os.path.join(test_image_dir, os.path.basename(img_path))
        copyfile(img_path, save_img_path)

print("Dataset split complete! Original images saved for training, validation, and test. Annotations saved for training and validation.")

/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Original image saved at /content/drive/MyDrive/KMUTT/AI/NoPoseAnnotated/images/train/noPos (204).jpg and annotation saved at /content/drive/MyDrive/KMUTT/AI/NoPoseAnnotated/labels/train/noPos (204).txt
Original image saved at /content/drive/MyDrive/KMUTT/AI/NoPoseAnnotated/images/train/noPos (123).jpg and annotation saved at /content/drive/MyDrive/KMUTT/AI/NoPoseAnnotated/labels/train/noPos (123).txt
Original image saved at /content/drive/MyDrive/KMUTT/AI/NoPoseAnnotated/images/train/noPos (27).jpg and annotation saved at /content/drive/MyDrive/KMUTT/AI/NoPoseAnnotated/labels/train/noPos (27).txt
Original image saved at /content/drive/MyDrive/KMUTT/AI/NoPoseAnnotated/images/train/noPos (14).jpg and annotation saved at /content/drive/MyDrive/KMUTT/AI/NoPoseAnnotated/labels/train/noPos (14).txt
Original image saved at /content/drive/MyDrive/KMUTT/AI/NoPoseAnnotated/images/train/noPos (40).jpg and annotation saved at /content/drive/MyDrive/KMUTT/AI/NoPoseAnnotated/labels/train/noPos (40).